In [4]:
import librosa
import numpy as np
import soundfile as sf
import os
from scipy.signal import butter, lfilter

# Define la ruta de la carpeta de los archivos de audio originales y la carpeta de destino
input_folder = 'D:/SEMESTRE 2-2023/INTELIGENCIA ARTIFICIAL 3 - SIS330/dataset_sonidos_cortos/union_bebes_riendo16bits_ayuda/'
output_folder = 'D:/SEMESTRE 2-2023/INTELIGENCIA ARTIFICIAL 3 - SIS330/dataset_sonidos_cortos/union_bebes_riendo16bits_ayuda_augmentation/'

# Crea la carpeta de destino si no existe
os.makedirs(output_folder, exist_ok=True)

# Funciones de data augmentation como antes...
def butter_bandpass(lowcut, highcut, sr, order=5):
    nyquist = 0.5 * sr
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Función para aplicar un filtro de paso de banda
def butter_bandpass_filter(data, lowcut, highcut, sr, order=5):
    b, a = butter_bandpass(lowcut, highcut, sr, order=order)
    y = lfilter(b, a, data)
    return y

def add_echo(data, sr, delay=0.2, decay=0.4):
    echo_sample_length = int(sr * delay)
    echo = np.zeros((len(data) + echo_sample_length,))
    echo[:len(data)] = data
    echo[echo_sample_length:] += decay * data
    return echo[:len(data)]

# Función para procesar todos los archivos en la carpeta
def process_all_files(input_folder, output_folder):
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.wav'):
            # Cargar el archivo de audio
            audio_path = os.path.join(input_folder, file_name)
            y, sr = librosa.load(audio_path, sr=None)

            # Aplicar data augmentation
            apply_data_augmentation(y, sr, file_name, output_folder)

def apply_data_augmentation(y, sr, file_name, output_folder):
    # 1. Añadir menos ruido blanco
    wn = np.random.randn(len(y))
    y_noise = y + 0.02 * wn  # Reducimos la amplitud del ruido

    # 2. Cambio de tono más extremo (pitch shifting)
    y_pitch_shift = librosa.effects.pitch_shift(y, sr, n_steps=6)  # Cambiamos el tono por 5 semitonos

    # 3. Cambio de velocidad menos extremo (time stretch) y mantener la duración
    y_stretch = librosa.effects.time_stretch(y, 0.8)  # Reducimos ligeramente la velocidad
    if len(y_stretch) > len(y):
        y_stretch = y_stretch[:len(y)]
    else:
        y_stretch = np.pad(y_stretch, (0, len(y) - len(y_stretch)), 'constant')

    # 4. Cambiar el volumen de forma más extrema
    y_loud = y * 3  # Triplicamos el volumen

    # 5. Reverberación (eco) más intensa
    y_reverb = add_echo(y, sr, delay=0.2, decay=0.3)   # Aumentamos el retraso y la decadencia del eco

    # 6. Filtro de paso de banda más restrictivo
    y_bandpass = butter_bandpass_filter(y, lowcut=1000, highcut=2000, sr=sr)  # Restringimos más el rango de frecuencias

    # Formatos de nombres de archivos aumentados
    aug_names = ['loud']

    # Archivos aumentados
    augmented_files = [y_loud]

    # Guardar los archivos con los nombres modificados
    for aug_name, aug_file in zip(aug_names, augmented_files):
        output_file_name = file_name.replace('.wav', f'_{aug_name}.wav')
        sf.write(os.path.join(output_folder, output_file_name), aug_file, sr)

# Procesar todos los archivos
process_all_files(input_folder, output_folder)

print(f'Los archivos aumentados se han guardado en: {output_folder}')


Los archivos aumentados se han guardado en: D:/SEMESTRE 2-2023/INTELIGENCIA ARTIFICIAL 3 - SIS330/dataset_sonidos_cortos/union_bebes_riendo16bits_ayuda_augmentation/
